# Import

In [3]:
import pandas as pd
import numpy as np
import random

# att48

## Function

In [442]:
### Get points
def get_points(data='data/att48.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int((float(list_data[i].split(' ')[1]))), int((float(list_data[i].split(' ')[2])))))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()
    

## Run

In [443]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 48
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []

list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)

        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
        print(f'Total distance : {total_distance}')



        # print(f'List total path : {list_paths}')
        # print(f'List total distance : {list_total_distance}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')
    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]

    ### Pair paths
    list_pair_paths = []
    list_pair_paths_distance = []
    for j in range(len(list(list_paths.values()))):
        pair_paths = []
        pair_paths_distance = []
        for k in range(len(list(list_paths.values()))):
            try:
                pair_paths.append([list(list_paths.values())[j][k], list(list_paths.values())[j][k+1]])
                pair_paths_distance.append(list(list_total_distance.values())[j])
                # print(f'Paths : {list(list_paths.values())[j][k]}, {list(list_paths.values())[j][k+1]} | Distance : {list(list_total_distance.values())[j]}')
            except:
                pass
        list_pair_paths.append(pair_paths)
        list_pair_paths_distance.append(pair_paths_distance)

    ### Calculation pair paths
    calculation_pair_paths = {}
    for i in range(len(list_pair_paths)):
        for j, k in zip(np.array(list_pair_paths)[i], np.array(list_pair_paths_distance[i])):
            if not str(j) in calculation_pair_paths:
                calculation_pair_paths[str(j)] = 1/k
            else:
                calculation_pair_paths[str(j)] += 1/k

    # Split str to int & update pheromone
    for i, j in zip(range(len(list(calculation_pair_paths.keys()))), list(calculation_pair_paths.values())):
        i = list(calculation_pair_paths.keys())[i].lstrip().split('[')[1].split(']')[0].split(' ')
        i_0 = i[0]
        if i_0 == '':
            i_0 = i[1]
        i_0 = int(i_0)
        i_1 = int(i[-1])
        # print(f'{i_0}, {i_1} : {j}')
        t0[i_0][i_1] += ((1-p) * t0[i_0][i_1]) + (j)

    # print(f'Pheromone after update : {sum(sum(t0))}')
    
    # # print(f'Shortest distance value: {shortest_distance_value}')
    # # print(f'Shortest distance index: {shortest_distance_index}')
    # # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)

print(f'List of shortest distance values : {list_shortest_distance_value}')

Gen : 0, Ant : 0
Paths : [4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 39, 8, 0, 7, 37, 30, 21, 15, 2, 33, 40, 28, 1, 41, 9, 23, 44, 34, 25, 3]
Total distance : 36447.7346856941
Gen : 0, Ant : 1
Paths : [12, 24, 13, 22, 10, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 46, 20, 38, 31, 47, 4, 28, 1, 41, 9, 23, 44, 34, 25, 3, 33, 2, 21, 15, 0, 8, 37, 30, 39, 7, 40]
Total distance : 38857.48438859677
Gen : 0, Ant : 2
Paths : [8, 0, 7, 37, 30, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 45, 32, 14, 11, 10, 22, 13, 24, 12, 20, 46, 19, 39, 2, 21, 15, 40, 33, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1]
Total distance : 35713.112998296514
Gen : 0, Ant : 3
Paths : [40, 33, 13, 24, 12, 22, 10, 11, 14, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36, 18, 26, 42, 16, 19, 46, 20, 38, 31, 47, 4, 28, 1, 41, 9, 23, 44, 34, 25, 3, 2, 21, 15, 0, 8, 37, 30, 39, 7]
Total distance : 38355.86967514288
Gen : 0, Ant : 4
Paths : [14

In [444]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2, 22, 10, 11, 14, 32, 45, 39, 19, 46, 20, 12, 24, 13, 33, 40, 28, 4, 47, 38, 31, 23, 9, 41, 25, 3, 34, 44, 1]': 35135.455097450664}

## DataFrame

In [445]:
df_att48 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_att48

,Gen,Ant,Distance,Path
0,0,0,36447.734686,"[4, 47, 38, 31, 20, 46, 10, 22, 13, 24, 12, 11..."
1,0,1,38857.484389,"[12, 24, 13, 22, 10, 11, 14, 32, 45, 43, 17, 6..."
2,0,2,35713.112998,"[8, 0, 7, 37, 30, 43, 17, 6, 27, 35, 29, 5, 36..."
3,0,3,38355.869675,"[40, 33, 13, 24, 12, 22, 10, 11, 14, 32, 45, 4..."
4,0,4,37239.225898,"[14, 11, 32, 45, 43, 17, 6, 27, 35, 29, 5, 36,..."
...,...,...,...,...
955,19,43,41018.912086,"[17, 6, 27, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2..."
956,19,44,41719.430152,"[42, 26, 18, 36, 5, 29, 27, 6, 17, 35, 43, 30,..."
957,19,45,40948.540500,"[27, 6, 17, 35, 43, 30, 37, 8, 0, 7, 21, 15, 2..."
958,19,46,35512.194488,"[18, 36, 5, 29, 26, 42, 16, 27, 6, 17, 35, 43,..."


## DataFrame [Shortest distace]

In [446]:
df_att48[df_att48.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
34,0,34,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
76,1,28,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
111,2,15,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
155,3,11,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
200,4,8,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
276,5,36,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
313,6,25,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
371,7,35,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
386,8,2,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
433,9,1,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."


## Save

In [472]:
!mkdir algorithm_v2

In [477]:
df_att48.to_pickle('algorithm_v2/df_att48.pkl')

In [485]:
t = pd.read_pickle('algorithm_v2/df_att48.pkl')
t[t.Distance == t.Distance.min()]

,Gen,Ant,Distance,Path
34,0,34,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
76,1,28,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
111,2,15,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
155,3,11,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
200,4,8,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
276,5,36,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
313,6,25,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
371,7,35,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
386,8,2,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."
433,9,1,35135.455097,"[16, 26, 18, 36, 5, 29, 42, 27, 6, 17, 35, 43,..."


# berlin52

## Function

In [447]:
### Get points
def get_points(data='data/berlin52.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int((float(list_data[i].split(' ')[1]))), int((float(list_data[i].split(' ')[2])))))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()
    

## Run

In [448]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 52
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []

list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)

        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
        print(f'Total distance : {total_distance}')



        # print(f'List total path : {list_paths}')
        # print(f'List total distance : {list_total_distance}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')
    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]

    ### Pair paths
    list_pair_paths = []
    list_pair_paths_distance = []
    for j in range(len(list(list_paths.values()))):
        pair_paths = []
        pair_paths_distance = []
        for k in range(len(list(list_paths.values()))):
            try:
                pair_paths.append([list(list_paths.values())[j][k], list(list_paths.values())[j][k+1]])
                pair_paths_distance.append(list(list_total_distance.values())[j])
                # print(f'Paths : {list(list_paths.values())[j][k]}, {list(list_paths.values())[j][k+1]} | Distance : {list(list_total_distance.values())[j]}')
            except:
                pass
        list_pair_paths.append(pair_paths)
        list_pair_paths_distance.append(pair_paths_distance)

    ### Calculation pair paths
    calculation_pair_paths = {}
    for i in range(len(list_pair_paths)):
        for j, k in zip(np.array(list_pair_paths)[i], np.array(list_pair_paths_distance[i])):
            if not str(j) in calculation_pair_paths:
                calculation_pair_paths[str(j)] = 1/k
            else:
                calculation_pair_paths[str(j)] += 1/k

    # Split str to int & update pheromone
    for i, j in zip(range(len(list(calculation_pair_paths.keys()))), list(calculation_pair_paths.values())):
        i = list(calculation_pair_paths.keys())[i].lstrip().split('[')[1].split(']')[0].split(' ')
        i_0 = i[0]
        if i_0 == '':
            i_0 = i[1]
        i_0 = int(i_0)
        i_1 = int(i[-1])
        # print(f'{i_0}, {i_1} : {j}')
        t0[i_0][i_1] += ((1-p) * t0[i_0][i_1]) + (j)

    # print(f'Pheromone after update : {sum(sum(t0))}')
    
    # # print(f'Shortest distance value: {shortest_distance_value}')
    # # print(f'Shortest distance index: {shortest_distance_index}')
    # # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)

print(f'List of shortest distance values : {list_shortest_distance_value}')

Gen : 0, Ant : 0
Paths : [12, 26, 27, 25, 46, 13, 51, 10, 50, 11, 24, 3, 5, 4, 14, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 49, 19, 22, 30, 17, 21, 0, 48, 31, 44, 18, 40, 7, 9, 8, 42, 32, 2, 16, 20, 29, 28, 41, 6, 1]
Total distance : 8115.435584058902
Gen : 0, Ant : 1
Paths : [18, 44, 31, 48, 0, 21, 30, 17, 2, 16, 20, 22, 19, 49, 15, 43, 33, 34, 35, 38, 39, 37, 36, 47, 23, 4, 14, 5, 3, 24, 45, 28, 29, 41, 6, 1, 40, 7, 9, 8, 42, 32, 50, 11, 27, 26, 25, 46, 12, 13, 51, 10]
Total distance : 8098.993270804586
Gen : 0, Ant : 2
Paths : [1, 6, 41, 20, 30, 17, 21, 0, 48, 31, 35, 34, 33, 38, 39, 37, 36, 47, 23, 4, 14, 5, 3, 24, 45, 43, 15, 49, 19, 22, 29, 28, 46, 25, 26, 27, 11, 50, 10, 51, 12, 13, 42, 9, 8, 7, 40, 18, 44, 2, 16, 32]
Total distance : 8714.828650738868
Gen : 0, Ant : 3
Paths : [3, 5, 4, 14, 23, 47, 37, 39, 36, 38, 35, 34, 33, 43, 45, 15, 49, 19, 22, 30, 17, 21, 0, 48, 31, 44, 18, 40, 7, 9, 8, 42, 24, 11, 27, 26, 25, 46, 12, 13, 51, 10, 50, 32, 2, 16, 20, 29, 28, 41, 6, 1]

In [449]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4, 14, 5, 3, 24, 11, 27, 26, 25, 46, 12, 13, 51, 10, 50, 32, 42, 9, 8, 7, 40, 18, 44, 31, 48, 0, 21, 30, 17, 2, 16, 20, 22, 19, 49, 15, 28, 29, 41, 6, 1]': 7310.669644787184}

## DataFrame

In [450]:
df_berlin52 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_berlin52

,Gen,Ant,Distance,Path
0,0,0,8115.435584,"[12, 26, 27, 25, 46, 13, 51, 10, 50, 11, 24, 3..."
1,0,1,8098.993271,"[18, 44, 31, 48, 0, 21, 30, 17, 2, 16, 20, 22,..."
2,0,2,8714.828651,"[1, 6, 41, 20, 30, 17, 21, 0, 48, 31, 35, 34, ..."
3,0,3,8411.815631,"[3, 5, 4, 14, 23, 47, 37, 39, 36, 38, 35, 34, ..."
4,0,4,8571.147041,"[36, 39, 37, 23, 47, 4, 14, 5, 3, 24, 45, 43, ..."
...,...,...,...,...
1035,19,47,8413.745787,"[30, 17, 21, 0, 48, 31, 35, 34, 33, 38, 39, 37..."
1036,19,48,8401.851818,"[17, 30, 21, 0, 48, 31, 35, 34, 33, 38, 39, 37..."
1037,19,49,8652.140765,"[16, 2, 17, 30, 21, 0, 48, 31, 35, 34, 33, 38,..."
1038,19,50,8284.899951,"[21, 0, 48, 31, 35, 34, 33, 38, 39, 37, 36, 47..."


## DataFrame [Shortest distace]

In [451]:
df_berlin52[df_berlin52.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
37,0,37,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
91,1,39,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
121,2,17,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
173,3,17,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
243,4,35,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
292,5,32,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
351,6,39,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
411,7,47,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
430,8,14,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
502,9,34,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."


## Save

In [486]:
df_berlin52.to_pickle('algorithm_v2/df_berlin52.pkl')

In [488]:
t = pd.read_pickle('algorithm_v2/df_berlin52.pkl')
t[t.Distance == t.Distance.min()]

,Gen,Ant,Distance,Path
37,0,37,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
91,1,39,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
121,2,17,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
173,3,17,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
243,4,35,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
292,5,32,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
351,6,39,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
411,7,47,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
430,8,14,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."
502,9,34,7310.669645,"[37, 39, 36, 38, 35, 34, 33, 43, 45, 47, 23, 4..."


# eil76

## Function

In [452]:
### Get points
def get_points(data='data/eil76.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int((float(list_data[i].split(' ')[1]))), int((float(list_data[i].split(' ')[2])))))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()
    

## Run

In [453]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 76
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []

list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)

        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
        print(f'Total distance : {total_distance}')



        # print(f'List total path : {list_paths}')
        # print(f'List total distance : {list_total_distance}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')
    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]

    ### Pair paths
    list_pair_paths = []
    list_pair_paths_distance = []
    for j in range(len(list(list_paths.values()))):
        pair_paths = []
        pair_paths_distance = []
        for k in range(len(list(list_paths.values()))):
            try:
                pair_paths.append([list(list_paths.values())[j][k], list(list_paths.values())[j][k+1]])
                pair_paths_distance.append(list(list_total_distance.values())[j])
                # print(f'Paths : {list(list_paths.values())[j][k]}, {list(list_paths.values())[j][k+1]} | Distance : {list(list_total_distance.values())[j]}')
            except:
                pass
        list_pair_paths.append(pair_paths)
        list_pair_paths_distance.append(pair_paths_distance)

    ### Calculation pair paths
    calculation_pair_paths = {}
    for i in range(len(list_pair_paths)):
        for j, k in zip(np.array(list_pair_paths)[i], np.array(list_pair_paths_distance[i])):
            if not str(j) in calculation_pair_paths:
                calculation_pair_paths[str(j)] = 1/k
            else:
                calculation_pair_paths[str(j)] += 1/k

    # Split str to int & update pheromone
    for i, j in zip(range(len(list(calculation_pair_paths.keys()))), list(calculation_pair_paths.values())):
        i = list(calculation_pair_paths.keys())[i].lstrip().split('[')[1].split(']')[0].split(' ')
        i_0 = i[0]
        if i_0 == '':
            i_0 = i[1]
        i_0 = int(i_0)
        i_1 = int(i[-1])
        # print(f'{i_0}, {i_1} : {j}')
        t0[i_0][i_1] += ((1-p) * t0[i_0][i_1]) + (j)

    # print(f'Pheromone after update : {sum(sum(t0))}')
    
    # # print(f'Shortest distance value: {shortest_distance_value}')
    # # print(f'Shortest distance index: {shortest_distance_index}')
    # # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)

print(f'List of shortest distance values : {list_shortest_distance_value}')

Gen : 0, Ant : 0
Paths : [50, 5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 62, 15, 2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, 58, 25, 11, 39, 16, 3, 29, 1, 0, 42, 40, 41, 63, 21, 60, 68, 35, 70, 59, 69, 19, 36, 4, 14, 56, 22, 55, 48, 23, 17, 49, 24, 54, 30]
Total distance : 619.7428490707621
Gen : 0, Ant : 1
Paths : [24, 49, 17, 23, 48, 15, 62, 32, 72, 61, 27, 73, 29, 1, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 44, 28, 47, 46, 20, 35, 70, 59, 69, 19, 36, 4, 14, 56, 3, 25, 11, 39, 16, 50, 5, 0, 42, 40, 41, 63, 21, 60, 68, 22, 55, 2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, 58, 30, 54]
Total distance : 641.0848775586056
Gen : 0, Ant : 2
Paths : [12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18, 53, 56, 14, 4, 28, 44, 3, 74, 75, 66, 25, 11, 39, 16, 50, 5, 67, 1, 29, 73, 27, 61, 72, 32, 62, 15, 2, 43, 31, 8, 38, 71, 57, 9, 37, 64, 10, 65, 58, 30, 24, 49, 17, 23, 48, 22, 55, 40, 41, 42, 0, 21, 60, 20, 46, 47, 35, 70

In [454]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[55, 22, 62, 15, 50, 5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12, 26, 51, 44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 0, 42, 40, 41, 63, 21, 60, 68, 35, 70, 59, 69, 19, 36, 4, 14, 56, 3, 29, 1, 16, 39, 11, 25, 57, 71, 38, 8, 31, 43, 2, 48, 23, 17, 49, 24, 54, 30, 9, 37, 64, 10, 65, 58]': 571.1464908173365,
 '[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18, 58, 10, 65, 64, 37, 9, 57, 71, 38, 8, 31, 43, 2, 15, 62, 32, 72, 61, 27, 73, 29, 1, 67, 74, 75, 66, 25, 11, 39, 16, 50, 5, 3, 44, 28, 47, 46, 20, 35, 70, 59, 69, 19, 36, 4, 14, 56, 68, 60, 21, 0, 42, 40, 41, 63, 55, 22, 48, 23, 17, 49, 24, 54, 30]': 568.8277446430413}

## DataFrame

In [455]:
df_eil76 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_eil76

,Gen,Ant,Distance,Path
0,0,0,619.742849,"[50, 5, 67, 74, 75, 66, 33, 45, 7, 34, 6, 52, ..."
1,0,1,641.084878,"[24, 49, 17, 23, 48, 15, 62, 32, 72, 61, 27, 7..."
2,0,2,647.169657,"[12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18, 53,..."
3,0,3,632.076139,"[32, 72, 61, 27, 73, 29, 1, 67, 74, 75, 66, 33..."
4,0,4,628.433218,"[44, 28, 47, 46, 20, 73, 27, 61, 72, 32, 62, 1..."
...,...,...,...,...
1515,19,71,613.468319,"[38, 8, 31, 43, 2, 15, 62, 32, 72, 61, 27, 73,..."
1516,19,72,614.898022,"[45, 33, 51, 26, 44, 28, 47, 46, 20, 73, 27, 6..."
1517,19,73,627.189708,"[25, 66, 33, 45, 7, 34, 6, 52, 13, 18, 53, 12,..."
1518,19,74,654.641790,"[36, 19, 69, 59, 70, 35, 46, 20, 73, 27, 61, 7..."


## DataFrame [Shortest distace]

In [456]:
df_eil76[df_eil76.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
146,1,70,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
213,2,61,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
249,3,21,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
344,4,40,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
420,5,40,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
490,6,34,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
541,7,9,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
675,8,67,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
746,9,62,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
770,10,10,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."


## Save

In [ ]:
df_eil76.to_pickle('algorithm_v2/df_eil76.pkl')

In [489]:
t = pd.read_pickle('algorithm_v2/df_eil76.pkl')
t[t.Distance == t.Distance.min()]

,Gen,Ant,Distance,Path
146,1,70,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
213,2,61,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
249,3,21,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
344,4,40,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
420,5,40,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
490,6,34,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
541,7,9,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
675,8,67,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
746,9,62,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."
770,10,10,568.827745,"[53, 12, 26, 51, 33, 45, 7, 34, 6, 52, 13, 18,..."


# kroA100

## Function

In [457]:
### Get points
def get_points(data='data/kroA100.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int((float(list_data[i].split(' ')[1]))), int((float(list_data[i].split(' ')[2])))))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()
    

## Run

In [458]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 100
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []

list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)

        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
        print(f'Total distance : {total_distance}')



        # print(f'List total path : {list_paths}')
        # print(f'List total distance : {list_total_distance}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')
    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]

    ### Pair paths
    list_pair_paths = []
    list_pair_paths_distance = []
    for j in range(len(list(list_paths.values()))):
        pair_paths = []
        pair_paths_distance = []
        for k in range(len(list(list_paths.values()))):
            try:
                pair_paths.append([list(list_paths.values())[j][k], list(list_paths.values())[j][k+1]])
                pair_paths_distance.append(list(list_total_distance.values())[j])
                # print(f'Paths : {list(list_paths.values())[j][k]}, {list(list_paths.values())[j][k+1]} | Distance : {list(list_total_distance.values())[j]}')
            except:
                pass
        list_pair_paths.append(pair_paths)
        list_pair_paths_distance.append(pair_paths_distance)

    ### Calculation pair paths
    calculation_pair_paths = {}
    for i in range(len(list_pair_paths)):
        for j, k in zip(np.array(list_pair_paths)[i], np.array(list_pair_paths_distance[i])):
            if not str(j) in calculation_pair_paths:
                calculation_pair_paths[str(j)] = 1/k
            else:
                calculation_pair_paths[str(j)] += 1/k

    # Split str to int & update pheromone
    for i, j in zip(range(len(list(calculation_pair_paths.keys()))), list(calculation_pair_paths.values())):
        i = list(calculation_pair_paths.keys())[i].lstrip().split('[')[1].split(']')[0].split(' ')
        i_0 = i[0]
        if i_0 == '':
            i_0 = i[1]
        i_0 = int(i_0)
        i_1 = int(i[-1])
        # print(f'{i_0}, {i_1} : {j}')
        t0[i_0][i_1] += ((1-p) * t0[i_0][i_1]) + (j)

    # print(f'Pheromone after update : {sum(sum(t0))}')
    
    # # print(f'Shortest distance value: {shortest_distance_value}')
    # # print(f'Shortest distance index: {shortest_distance_index}')
    # # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)

print(f'List of shortest distance values : {list_shortest_distance_value}')

Gen : 0, Ant : 0
Paths : [92, 27, 66, 57, 60, 24, 80, 68, 72, 49, 43, 1, 53, 39, 63, 67, 84, 38, 29, 95, 77, 51, 4, 36, 32, 75, 12, 94, 81, 47, 99, 70, 40, 13, 2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 56, 6, 8, 86, 50, 76, 59, 61, 22, 97, 90, 44, 31, 10, 14, 16, 73, 20, 58, 71, 9, 83, 35, 37, 23, 17, 78, 52, 87, 15, 21, 93, 69, 65, 64, 3, 96, 55, 79, 30, 88, 41, 7, 91, 0, 62, 5, 48, 89, 18, 74, 25, 98, 46, 42]
Total distance : 26499.090437654966
Gen : 0, Ant : 1
Paths : [4, 36, 77, 51, 95, 38, 29, 47, 99, 70, 40, 13, 2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 56, 6, 8, 86, 50, 60, 24, 80, 68, 72, 49, 43, 1, 53, 39, 63, 67, 84, 81, 94, 12, 32, 75, 42, 61, 59, 76, 22, 97, 90, 44, 31, 10, 14, 16, 73, 20, 58, 71, 9, 83, 35, 37, 23, 17, 78, 52, 87, 15, 21, 93, 69, 65, 64, 3, 96, 55, 79, 30, 88, 41, 7, 91, 0, 62, 5, 48, 89, 18, 74, 25, 98, 46, 92, 27, 66, 57]
Total distance : 26334.207744046664
Gen : 0, Ant : 2
Paths : [55, 96, 74, 18, 52, 78, 17, 23, 37, 35, 98, 83, 9, 71, 20, 73, 58, 16,

In [459]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37, 35, 98, 83, 9, 71, 20, 73, 58, 16, 14, 10, 31, 90, 97, 22, 44, 46, 62, 5, 48, 89, 78, 52, 18, 74, 96, 55, 79, 30, 88, 41, 7, 91, 0, 92, 27, 66, 57, 60, 24, 80, 68, 72, 49, 43, 1, 53, 39, 63, 67, 84, 38, 29, 95, 77, 51, 4, 36, 32, 75, 12, 94, 81, 47, 99, 70, 40, 13, 2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 56, 6, 8, 86, 50, 76, 59, 61, 42]': 23137.514597468155}

## DataFrame

In [460]:
df_kroA100 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_kroA100

,Gen,Ant,Distance,Path
0,0,0,26499.090438,"[92, 27, 66, 57, 60, 24, 80, 68, 72, 49, 43, 1..."
1,0,1,26334.207744,"[4, 36, 77, 51, 95, 38, 29, 47, 99, 70, 40, 13..."
2,0,2,24741.564864,"[55, 96, 74, 18, 52, 78, 17, 23, 37, 35, 98, 8..."
3,0,3,25496.044720,"[20, 73, 58, 71, 9, 83, 35, 37, 23, 17, 78, 52..."
4,0,4,26359.139696,"[46, 31, 10, 14, 16, 73, 20, 58, 71, 9, 83, 35..."
...,...,...,...,...
1995,19,95,24783.114036,"[50, 86, 8, 6, 56, 19, 11, 26, 85, 34, 61, 59,..."
1996,19,96,27085.738004,"[35, 37, 23, 17, 78, 52, 87, 15, 21, 93, 69, 6..."
1997,19,97,25460.928607,"[2, 45, 28, 33, 82, 54, 11, 26, 85, 34, 19, 56..."
1998,19,98,24176.995331,"[44, 90, 97, 22, 59, 61, 76, 19, 11, 26, 85, 3..."


## DataFrame [Shortest distace]

In [461]:
df_kroA100[df_kroA100.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
46,0,46,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
168,1,68,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
281,2,81,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
356,3,56,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
407,4,7,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
590,5,90,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
675,6,75,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
750,7,50,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
813,8,13,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
964,9,64,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."


## Save

In [ ]:
df_kroA100.to_pickle('algorithm_v2/df_kroA100.pkl')

In [490]:
t = pd.read_pickle('algorithm_v2/df_kroA100.pkl')
t[t.Distance == t.Distance.min()]

,Gen,Ant,Distance,Path
46,0,46,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
168,1,68,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
281,2,81,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
356,3,56,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
407,4,7,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
590,5,90,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
675,6,75,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
750,7,50,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
813,8,13,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."
964,9,64,23137.514597,"[3, 64, 65, 25, 69, 21, 15, 87, 93, 17, 23, 37..."


# pr76

## Function

In [462]:
### Get points
def get_points(data='data/pr76.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int((float(list_data[i].split(' ')[1]))), int((float(list_data[i].split(' ')[2])))))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()
    

## Run

In [463]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 76
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []

list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)

        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
        print(f'Total distance : {total_distance}')



        # print(f'List total path : {list_paths}')
        # print(f'List total distance : {list_total_distance}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')
    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]

    ### Pair paths
    list_pair_paths = []
    list_pair_paths_distance = []
    for j in range(len(list(list_paths.values()))):
        pair_paths = []
        pair_paths_distance = []
        for k in range(len(list(list_paths.values()))):
            try:
                pair_paths.append([list(list_paths.values())[j][k], list(list_paths.values())[j][k+1]])
                pair_paths_distance.append(list(list_total_distance.values())[j])
                # print(f'Paths : {list(list_paths.values())[j][k]}, {list(list_paths.values())[j][k+1]} | Distance : {list(list_total_distance.values())[j]}')
            except:
                pass
        list_pair_paths.append(pair_paths)
        list_pair_paths_distance.append(pair_paths_distance)

    ### Calculation pair paths
    calculation_pair_paths = {}
    for i in range(len(list_pair_paths)):
        for j, k in zip(np.array(list_pair_paths)[i], np.array(list_pair_paths_distance[i])):
            if not str(j) in calculation_pair_paths:
                calculation_pair_paths[str(j)] = 1/k
            else:
                calculation_pair_paths[str(j)] += 1/k

    # Split str to int & update pheromone
    for i, j in zip(range(len(list(calculation_pair_paths.keys()))), list(calculation_pair_paths.values())):
        i = list(calculation_pair_paths.keys())[i].lstrip().split('[')[1].split(']')[0].split(' ')
        i_0 = i[0]
        if i_0 == '':
            i_0 = i[1]
        i_0 = int(i_0)
        i_1 = int(i[-1])
        # print(f'{i_0}, {i_1} : {j}')
        t0[i_0][i_1] += ((1-p) * t0[i_0][i_1]) + (j)

    # print(f'Pheromone after update : {sum(sum(t0))}')
    
    # # print(f'Shortest distance value: {shortest_distance_value}')
    # # print(f'Shortest distance index: {shortest_distance_index}')
    # # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)

print(f'List of shortest distance values : {list_shortest_distance_value}')

Gen : 0, Ant : 0
Paths : [66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 2, 3, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 73, 37, 38, 32, 31, 25, 26, 48, 49, 70, 71, 72, 69, 75, 74]
Total distance : 134541.60654461547
Gen : 0, Ant : 1
Paths : [61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 2, 3, 4, 19, 18, 30, 29, 28, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 70, 71, 72, 49, 48, 47, 43, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 25, 26, 32, 31, 37, 38, 46, 68, 67, 66, 69, 73]
Total distance : 138814.18258133676
Gen : 0, Ant : 2
Paths : [0, 1, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46, 68, 67, 66, 49, 48, 51, 52, 53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 36, 35, 34, 33, 39, 40, 59, 58, 57, 56, 62, 63, 61, 60, 54, 55, 50, 65, 64, 70, 71, 72, 38, 37, 31, 32

In [464]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 2, 3, 4, 19, 18, 30, 29, 28, 25, 26, 43, 47, 46, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73, 37, 38, 48, 49, 66, 67, 68, 69, 70, 71, 72]': 123389.07045571336}

## DataFrame

In [465]:
df_pr76 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_pr76

,Gen,Ant,Distance,Path
0,0,0,134541.606545,"[66, 67, 68, 46, 47, 43, 44, 45, 23, 24, 20, 2..."
1,0,1,138814.182581,"[61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 1..."
2,0,2,137192.626267,"[0, 1, 22, 21, 20, 24, 23, 45, 44, 43, 47, 46,..."
3,0,3,127515.083716,"[9, 8, 5, 6, 7, 2, 3, 4, 19, 18, 30, 29, 28, 2..."
4,0,4,125338.613630,"[12, 13, 11, 10, 16, 17, 36, 35, 34, 33, 39, 4..."
...,...,...,...,...
1515,19,71,138630.867330,"[23, 24, 20, 21, 22, 0, 1, 2, 3, 4, 19, 18, 30..."
1516,19,72,133742.965541,"[53, 41, 42, 27, 28, 29, 30, 18, 19, 4, 5, 6, ..."
1517,19,73,135213.678277,"[70, 71, 72, 63, 62, 56, 57, 58, 59, 40, 39, 3..."
1518,19,74,127319.936260,"[3, 2, 5, 6, 7, 8, 9, 4, 19, 18, 30, 29, 28, 2..."


## DataFrame [Shortest distace]

In [466]:
df_pr76[df_pr76.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
74,0,74,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
88,1,12,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
199,2,47,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
303,3,75,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
363,4,59,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
431,5,51,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
476,6,20,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
575,7,43,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
655,8,47,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
687,9,3,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."


## Save

In [ ]:
df_pr76.to_pickle('algorithm_v2/df_pr76.pkl')

In [491]:
t = pd.read_pickle('algorithm_v2/df_pr76.pkl')
t[t.Distance == t.Distance.min()]

,Gen,Ant,Distance,Path
74,0,74,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
88,1,12,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
199,2,47,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
303,3,75,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
363,4,59,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
431,5,51,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
476,6,20,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
575,7,43,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
655,8,47,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."
687,9,3,123389.070456,"[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 5..."


In [16]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[31, 32, 27, 42, 41, 53, 52, 51, 50, 65, 64, 55, 54, 57, 56, 62, 63, 61, 60, 58, 59, 40, 39, 33, 34, 35, 36, 17, 16, 10, 11, 12, 13, 14, 15, 9, 8, 5, 6, 7, 2, 3, 4, 19, 18, 30, 29, 28, 25, 26, 43, 47, 46, 44, 45, 23, 24, 20, 21, 22, 0, 1, 74, 75, 73, 37, 38, 48, 49, 66, 67, 68, 69, 70, 71, 72]': 123389.07045571336}

# rd100

## Function

In [467]:
### Get points
def get_points(data='data/rd100.txt'):
    with open(data) as f:
        line = f.readlines()
        list_data = []
        for index, data in enumerate(line):
            list_data.append(data.strip())

    points = []
    for i in range(len(list_data)):
        points.append((int((float(list_data[i].split(' ')[1]))), int((float(list_data[i].split(' ')[2])))))
    return points

### Get distance
def distance(point_i : int, point_j: int):
    return ((point_i[0] - point_j[0])**2 + (point_i[1] - point_j[1])**2)**(1/2)

def get_cost_matrix():
    cost_matrix = []
    for i in range(all_points):
        row = []
        for j in range(all_points):
            row.append(distance(get_points()[i], get_points()[j]))
        cost_matrix.append(row)
    return np.array(cost_matrix)


### DataFrame
def createDF(gen, ant, distance, paht):
    df = pd.DataFrame({'Gen' : gen, 'Ant' : ant,  'Distance' : distance, 'Path' : paht})
    return df

### Reset values
def reset_values():
    get_cost_matrix()
    

## Run

In [468]:
### Default values
all_points = len(get_points())
cost_matrix = get_cost_matrix() # Distance between i & j
cm = cost_matrix.copy()
n = np.array([[0 if i == j else 1 / cost_matrix[i][j] for i in range(all_points)] for j in range(all_points)]) # 1/distance
gens= 20
ants= 100
alpha= 3
beta= 3
p= .02

t0 = np.array([[0 if i==j else 1 for i in range(all_points)] for j in range(all_points)], dtype=float) # Default pheromone

list_gens_for_df = []
list_ants_for_df = []
list_total_distance_for_df = []
list_paths_for_df = []

list_shortest_distance_value = []
list_shortest_distance_index = []
list_shortest_path = []
for gen in range(gens):

    list_paths = {}
    list_total_distance = {}

    random_start = list(range(all_points))
    for ant in range(ants):
        print(f'Gen : {gen}, Ant : {ant}')
        list_gens_for_df.append(gen)
        list_ants_for_df.append(ant)

        allowed = [i for i in np.arange(all_points)]
        paths = []

        ### -------------------------------------------------- Start --------------------------------------------------
        start = random_start.pop(random.randrange(len(random_start))) # Random between 0 to 47
        selected = start

        paths.append(selected)

        # print(f'Start : {selected}')
        best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[1])[0][0] # Return array, # sorted(cost_matrix[paths[-1]])[1] : ค่าที่ใกล้สุด
        distance_ij = sorted(cost_matrix[paths[-1]])[1] # index[0] : 0
        # print(f'Best paths : {best_path} ,Distance : {distance_ij}')
        # print(paths)
        ### Recheck best path
        # for best in enumerate(cost_matrix[paths[-1]]):
        #     print(best)

        allowed.remove(selected)
        for i in range(all_points):
            cost_matrix[selected][i] = 0
            cost_matrix[i][selected] = 0

        ### -------------------------------------------------- Denominator --------------------------------------------------
        list_t0 = []
        list_n = []
        for i in allowed:
            # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
            # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
            list_t0.append(t0[selected][i] ** alpha)
            list_n.append(n[selected][i] ** beta)
        list_t0 = np.array(list_t0)
        list_n = np.array(list_n)
        denominator = sum(list_t0 * list_n)
        # print(f'Denominator : {denominator}')
        # print('-'*50)
        
        ### -------------------------------------------------- Probability --------------------------------------------------
        for city in range(len(allowed)):
            prob = {}
            for i in allowed:
                prob.update({i: (t0[paths[-1]][i]**alpha) * (n[paths[-1]][i]**beta) / denominator})
            max_prob = sorted(prob.values())[-1]
            selected = list(prob.keys())[list(prob.values()).index(max_prob)]

            # for key, value in zip(prob.keys(), prob.values()):
            #     print(key, value)

            paths.append(selected)

            # print(f'Allowed : {allowed}')
            # print(f'Select : {selected}')

            ### Check walked path
            walked = 0
            for i in cost_matrix[paths[-1]]:
                if i == 0:
                    walked += 1
            # print(f'walked : {walked}')
            if walked != all_points:
                best_path = np.where(cost_matrix[paths[-1]] == sorted(cost_matrix[paths[-1]])[walked])[0][0]
                distance_ij = sorted(cost_matrix[paths[-1]])[walked]
                # print(f'Best paths : {best_path} ,Distance : {distance_ij}')

            # print(f'Path : {paths}')
            ### Recheck best path
            # for best in enumerate(cost_matrix[paths[-1]]):
            #     print(best)

            ### Remove the path that has been walked
            allowed.remove(selected)
            for i in range(all_points):
                cost_matrix[selected][i] = 0
                cost_matrix[i][selected] = 0

            ### -------------------------------------------------- Denominator --------------------------------------------------
            list_t0 = []
            list_n = []
            for i in allowed:
                # print(f'Proromone [{selected}][{i}] : {t0[selected][i]} ** {alpha} = {t0[selected][i] ** alpha}')
                # print(f'n [{selected}][{i}] : {n[selected][i]} ** {beta} = {n[selected][i] ** beta}')
                list_t0.append(t0[selected][i] ** alpha)
                list_n.append(n[selected][i] ** beta)
            list_t0 = np.array(list_t0)
            list_n = np.array(list_n)
            denominator = sum(list_t0 * list_n)
            # print(f'Denominator : {denominator}')
            # print('-'*50)

        list_paths.update({ant : paths})
        list_paths_for_df.append(paths)
        print(f'Paths : {paths}')

        ### -------------------------------------------------- Total distance --------------------------------------------------
        total_distance = 0
        for i in range(len(paths)):
            if i != len(paths)-1:
                total_distance += (cm[paths[i]][paths[i+1]])

        list_total_distance.update({ant : total_distance})
        list_total_distance_for_df.append(total_distance)
        print(f'Total distance : {total_distance}')



        # print(f'List total path : {list_paths}')
        # print(f'List total distance : {list_total_distance}')

        ### -------------------------------------------------- Reset cost matrix --------------------------------------------------
        reset_values()

        # print(f'Pheromone before update : {sum(sum(t0))}')
    ### -------------------------------------------------- Update pheromone --------------------------------------------------
    shortest_distance_value = sorted(list_total_distance.values())[0]
    shortest_distance_index = list(list_total_distance.keys())[list(list_total_distance.values()).index(shortest_distance_value)]
    shortest_path = list_paths[shortest_distance_index]

    ### Pair paths
    list_pair_paths = []
    list_pair_paths_distance = []
    for j in range(len(list(list_paths.values()))):
        pair_paths = []
        pair_paths_distance = []
        for k in range(len(list(list_paths.values()))):
            try:
                pair_paths.append([list(list_paths.values())[j][k], list(list_paths.values())[j][k+1]])
                pair_paths_distance.append(list(list_total_distance.values())[j])
                # print(f'Paths : {list(list_paths.values())[j][k]}, {list(list_paths.values())[j][k+1]} | Distance : {list(list_total_distance.values())[j]}')
            except:
                pass
        list_pair_paths.append(pair_paths)
        list_pair_paths_distance.append(pair_paths_distance)

    ### Calculation pair paths
    calculation_pair_paths = {}
    for i in range(len(list_pair_paths)):
        for j, k in zip(np.array(list_pair_paths)[i], np.array(list_pair_paths_distance[i])):
            if not str(j) in calculation_pair_paths:
                calculation_pair_paths[str(j)] = 1/k
            else:
                calculation_pair_paths[str(j)] += 1/k

    # Split str to int & update pheromone
    for i, j in zip(range(len(list(calculation_pair_paths.keys()))), list(calculation_pair_paths.values())):
        i = list(calculation_pair_paths.keys())[i].lstrip().split('[')[1].split(']')[0].split(' ')
        i_0 = i[0]
        if i_0 == '':
            i_0 = i[1]
        i_0 = int(i_0)
        i_1 = int(i[-1])
        # print(f'{i_0}, {i_1} : {j}')
        t0[i_0][i_1] += ((1-p) * t0[i_0][i_1]) + (j)

    # print(f'Pheromone after update : {sum(sum(t0))}')
    
    # # print(f'Shortest distance value: {shortest_distance_value}')
    # # print(f'Shortest distance index: {shortest_distance_index}')
    # # print(f'Shortest path: {shortest_path}')
    list_shortest_distance_value.append(shortest_distance_value)
    list_shortest_distance_index.append(shortest_distance_index)
    list_shortest_path.append(shortest_path)

print(f'List of shortest distance values : {list_shortest_distance_value}')

Gen : 0, Ant : 0
Paths : [8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84, 11, 13, 86, 61, 17, 0, 59, 7, 68, 14, 62, 96, 85, 66, 12, 2, 32, 9, 78, 5, 52, 98, 46, 56, 35, 63, 90, 64, 79, 80, 29, 47, 82, 50, 89, 67, 70, 4, 60, 10, 33, 6, 39, 41, 42, 24, 23, 40, 21, 54, 38, 95, 28, 34, 43, 99, 51, 44, 15, 97, 65, 75, 57, 87, 83, 30, 22, 1, 88, 58, 94, 76, 92, 27, 36, 18, 45, 49, 72, 55, 91, 26, 16, 71, 69, 37, 53, 93]
Total distance : 9093.716611194877
Gen : 0, Ant : 1
Paths : [98, 52, 5, 78, 9, 32, 2, 3, 31, 77, 19, 73, 25, 8, 11, 84, 13, 81, 74, 20, 48, 66, 96, 62, 85, 14, 61, 17, 0, 59, 7, 68, 89, 50, 82, 67, 70, 86, 12, 63, 35, 56, 46, 93, 97, 65, 75, 57, 87, 83, 30, 15, 44, 51, 99, 43, 28, 34, 95, 38, 54, 39, 6, 33, 40, 21, 4, 60, 10, 80, 79, 64, 47, 29, 90, 41, 42, 24, 23, 22, 1, 88, 58, 94, 76, 92, 27, 36, 18, 45, 49, 72, 55, 91, 26, 16, 71, 69, 37, 53]
Total distance : 9616.906786048174
Gen : 0, Ant : 2
Paths : [68, 7, 59, 0, 17, 61, 86, 14, 62, 96, 85, 66, 12, 48, 20, 74, 81, 84, 11

In [469]:
list_shortest_distance_value_path = {f'{path}' : distance for path, distance in zip(list_shortest_path, list_shortest_distance_value)}
list_shortest_distance_value_path

{'[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84, 11, 13, 86, 61, 17, 0, 59, 7, 68, 14, 62, 96, 85, 66, 12, 2, 32, 9, 78, 5, 52, 98, 46, 56, 35, 63, 90, 64, 79, 80, 29, 47, 82, 50, 89, 67, 70, 4, 60, 10, 33, 6, 39, 41, 42, 24, 23, 40, 21, 54, 38, 95, 28, 34, 43, 99, 51, 44, 15, 97, 65, 75, 57, 87, 83, 30, 22, 1, 88, 58, 94, 76, 92, 27, 36, 18, 45, 49, 72, 55, 91, 26, 16, 71, 69, 37, 53, 93]': 9093.716611194877}

## DataFrame

In [470]:
df_rd100 = createDF(list_gens_for_df, list_ants_for_df, list_total_distance_for_df, list_paths_for_df)
df_rd100

,Gen,Ant,Distance,Path
0,0,0,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
1,0,1,9616.906786,"[98, 52, 5, 78, 9, 32, 2, 3, 31, 77, 19, 73, 2..."
2,0,2,9288.587858,"[68, 7, 59, 0, 17, 61, 86, 14, 62, 96, 85, 66,..."
3,0,3,9647.865412,"[40, 21, 4, 60, 10, 33, 6, 39, 41, 42, 24, 23,..."
4,0,4,9119.081870,"[77, 19, 73, 25, 8, 31, 3, 74, 20, 48, 81, 84,..."
...,...,...,...,...
1995,19,95,9484.410638,"[54, 38, 95, 28, 34, 43, 99, 51, 44, 15, 97, 6..."
1996,19,96,9275.697167,"[82, 50, 89, 64, 79, 80, 29, 47, 67, 70, 7, 59..."
1997,19,97,9267.806173,"[0, 59, 7, 68, 14, 62, 96, 85, 66, 12, 48, 20,..."
1998,19,98,10143.449417,"[37, 71, 16, 69, 53, 72, 49, 45, 55, 18, 36, 2..."


## DataFrame [Shortest distace]

In [471]:
df_rd100[df_rd100.Distance == list_shortest_distance_value[-1]]

,Gen,Ant,Distance,Path
0,0,0,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
151,1,51,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
251,2,51,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
337,3,37,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
426,4,26,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
503,5,3,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
623,6,23,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
713,7,13,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
815,8,15,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
982,9,82,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."


## Save

In [ ]:
df_rd100.to_pickle('algorithm_v2/df_rd100.pkl')

In [492]:
t = pd.read_pickle('algorithm_v2/df_rd100.pkl')
t[t.Distance == t.Distance.min()]

,Gen,Ant,Distance,Path
0,0,0,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
151,1,51,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
251,2,51,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
337,3,37,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
426,4,26,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
503,5,3,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
623,6,23,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
713,7,13,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
815,8,15,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
982,9,82,9093.716611,"[8, 25, 73, 19, 77, 31, 3, 74, 20, 48, 81, 84,..."
